In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import fiona
import requests
import json
import geopandas as gpd
from shapely.geometry import shape

# 1️⃣ 读取几何并确保其为符合 ohsome 要求的格式
shp_path = "/content/drive/MyDrive/Wuhan/Boundary.shp"
with fiona.open(shp_path) as src:
    # 获取第一个要素
    feature = next(iter(src))
    geom = shape(feature["geometry"])

    # 构建 FeatureCollection 格式，这是 API 最喜欢的格式
    aoi_geojson = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "geometry": geom.__geo_interface__,
            "properties": {}
        }]
    }

In [9]:
!pip install geopandas shapely fiona requests

In [12]:
url = "https://api.ohsome.org/v1/elements/geometry"
filter_expr = """
(
  leisure=park or
  leisure=garden or
  leisure=nature_reserve or
  leisure=recreation_ground or

  natural=wood or
  natural=grassland or
  natural=scrub or
  natural=heath or

  landuse=grass or
  landuse=forest or
  landuse=meadow or
  landuse=village_green or
  landuse=cemetery or
  landuse=allotments
)
and geometry:polygon
"""

payload = {
    # 注意：必须是字符串
    "bpolys": json.dumps(aoi_geojson),
    # 时间点（不是时间段）
    "time": "2020-09-01",
    "filter": filter_expr,
    # 返回 tags，方便后续语义筛选 / QA
    "properties": "tags"
}

In [13]:
try:
    # 使用 data=payload 以 form-data 形式发送，这是 ohsome 的标准要求
    response = requests.post(url, data=payload)

    # 如果发生 400 错误，打印服务器返回的具体错误信息（非常重要！）
    if response.status_code != 200:
        print(f"Server Response: {response.text}")

    response.raise_for_status()
    data = response.json()
    print("Successfully fetched data!")

except requests.exceptions.HTTPError as err:
    print(f"HTTP Error: {err}")

Successfully fetched data!


In [14]:
if data and 'features' in data:
    # 从 GeoJSON 特征集合创建
    gdf = gpd.GeoDataFrame.from_features(data['features'])
    # 设置原始坐标系（ohsome 默认返回 WGS84）
    gdf.set_crs(epsg=4326, inplace=True)

    if gdf.empty:
        print("警告：API 返回数据为空，没有要素可保存。")
    else:
        gdf = gdf.reset_index(drop=True)
        if 'tags' in gdf.columns:
            # 将字典转为 JSON 字符串
            gdf['tags_str'] = gdf['tags'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else str(x))
            # 删除原始字典列
            gdf = gdf.drop(columns=['tags'])
        for col in gdf.columns:
            if gdf[col].dtype == 'object' and col != 'geometry':
                gdf[col] = gdf[col].astype(str)
        output_filename = "All_green.shp"
        try:
            gdf.to_file(output_filename, driver='ESRI Shapefile', encoding='utf-8')
            print(f"成功保存到: {output_filename}")
        except Exception as e:
            print(f"保存失败: {e}")
else:
    print("错误：未获取到有效数据。")

成功保存到: All_green.shp


/tmp/ipython-input-2874059928.py:21: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_filename, driver='ESRI Shapefile', encoding='utf-8')
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:733: RuntimeWarning: Normalized/laundered field name: '@snapshotTimestamp' to '@snapshotT'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:733: RuntimeWarning: Normalized/laundered field name: 'name:de' to 'name_de'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:733: RuntimeWarning: Normalized/laundered field name: 'name:en' to 'name_en'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:733: RuntimeWarning: Normalized/laundered field name: 'designation' to 'designatio'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:733: RuntimeWarning: Normalized/laundered field name: 'addr:street' to 'addr_stree'
  ogr_write(
/usr/local/lib/python3.12/dist-packa